## 指定したheadの[Query,Key,Valuse]をマスクまたはスケーリングしたモデルを保存するコード

In [ ]:
from transformers import AutoTokenizer
from lib.utils.custommodel import CustomLlamaModelForCausalLM
from lib.utils.batch_inference import surgery
import json
import os
import torch
model_path = "Llama-2-7b-chat-hf"
# model_path = "Llama3.1-8B-Introduction"
# model_path = "Llama3.2-3B-Introduction"

if("/" in model_path):
    model_name = model_path.split("/")[-1]

### ハイパラ
* mask_type : [scale_mask / mean_mask]
    スケールマスク :　ヘッドのq,k,vのいずれかの重みにscale_factorをかけて調整，論文では小さい値をかけた無効化してた
    ミーンマスク   :　重みの平均を取る
* scale_factor : scale_maskの時q,k,vのいずれかの重みと積をとる値
* head_mask{} : (Head, Layer):mask_qkv = ['q','k','v']
    (3,5):['q','k','v]のとき，3層目のhead5のq,k,v全てを調整する
* name_prefix : 調整後のモデルの名前のprefix

In [ ]:
mask_qkv = ['q']
scale_factor = 1e-10
mask_type = "scale_mask"

head_mask = {
    # (Head, Layer): mask_qkv = ['q']
    (3,4): ['q','k','v']
    }
name_prefix = "run_test"


In [ ]:
model = CustomLlamaModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
surgery_path = f"./models/{model_name}-{name_prefix}"
# storage_path
surgery_model = surgery(model, 
                        tokenizer, 
                        head_mask, 
                        mask_type, 
                        scale_factor,
                        surgery_path)

### 調整したモデルでの推論

In [ ]:
from custom_lllama_predict import Custom_llama
model = Custom_llama(surgery_path,quant_type=4,use_sys_prompt=False,use_history=False)
generation_config = {
    "do_sample": True,
    "top_p": 0.95,
    "temperature": 0.7,
    "max_new_tokens": 2048
}
print(model.predict("あなたは誰ですか？",generation_config))



### データセットでモデルの検証

In [ ]:
from lib.utils.batch_inference import inference
from transformers import AutoTokenizer
from lib.utils.custommodel import CustomLlamaModelForCausalLM
import json
import os
import torch
import accelerate
model = CustomLlamaModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

### inference関数
##### ARGS
* surgery_path :読み込むモデルのパス
* data_path : 使うデータセットのパス
* accelerator : バッチ推論用accelerator
* generate_cfg :　モデルの推論用の引数
* inference_cgf :　torchのDataLoaderに渡すconfig
* seed :　再現性担保用のseed値．pythonとcudaとかのseedをまとめて設定
* sample_times : データセットの中の何個まで推論するか

In [ ]:
surgery_path = f"./models/{model_name}-{name_prefix}"
data = "jailbreakbench"
data_path = f"./exp_data/{data}.csv"
accelerator = accelerate.Accelerator()

generate_config = {
    "max_new_tokens": 256,
    "top_k": 1,
    "top_p": 0.9,
    "do_sample": False,
}
inference_config = {
    "use_conv": True,
    "store_path": f"./exp_res/{data}/",
    "batch_size": 50,
}

In [ ]:
inference(model_name=surgery_path,
          data_path=data_path,
          accelerator=accelerator,
          generate_cfg=generate_config,
          inference_cfg=inference_config,
          seed=32,
          sample_times=100)